# Warm Up
Question 1: how much AirBNB homes are earning in certain time frames and areas
## checking the data

In [1]:
import pandas as pd

In [27]:
df_calendar = pd.read_csv('boston/calendar.csv')

In [28]:
df_calendar.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [4]:
df_calendar.shape

(1308890, 4)

In [3]:
df_listings = pd.read_csv('boston/listings.csv')

In [4]:
df_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [7]:
df_listings.shape

(3585, 95)

In [5]:
df_reviews = pd.read_csv('boston/reviews.csv')

In [6]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [10]:
df_reviews.shape

(68275, 6)

###  Let's check if the listing id are same 

In [35]:
listing_id_set0 = set(df_listings['id'].value_counts().index.tolist())

In [36]:
len(listing_id_set0)

3585

In [37]:
listing_id_set1=set(df_calendar['listing_id'].value_counts().index.tolist())

In [38]:
len(listing_id_set1)

3585

In [39]:
listing_id_set2=set(df_reviews['listing_id'].value_counts().index.tolist())

In [40]:
len(listing_id_set2)

2829

In [41]:
listing_id_set0==listing_id_set1 # all calendar listing_id is same as in listings

True

In [43]:
listing_id_set2 | listing_id_set0 == listing_id_set0 # all listing id in reviews belong be listing id

True

### calculate the earning

In [54]:
df_calendar.dtypes # we need change the price to float

listing_id     int64
date          object
available     object
price         object
dtype: object

In [56]:
df_calendar['available'].value_counts(dropna=False)

f    665853
t    643037
Name: available, dtype: int64

In [67]:
#check consistance
df_calendar['price'][df_calendar['available']=='f'].isnull().mean() #should be all NaN

1.0

In [68]:
df_calendar['price'][df_calendar['available']=='t'].isnull().mean() #should all none NaN

0.0

Price is object, need to change to float

In [73]:
df_calendar['price'].dropna().str.contains("\$*").mean() #check if all valid price starting with $

1.0

In [7]:
float(df_calendar['price'][365][1:])

65.0

In [74]:
df_calendar['listing_id'].isnull().mean() # make sure all listing_id is valid

0.0

In [29]:
df_calendar['price'].fillna('$0',inplace=True)#fill all NaN by $0

In [30]:
df_calendar['price'].head()

0    $0
1    $0
2    $0
3    $0
4    $0
Name: price, dtype: object

In [33]:
def drop_c(price):
    res=""
    for c in price:
        if c !='$' and c!=',':
            res += c
    return res

0    $0
1    $0
2    $0
3    $0
4    $0
Name: price, dtype: object

In [34]:
df_calendar['price']=df_calendar['price'].apply(drop_c)#drop all $ and ,

In [35]:
df_calendar['price'].head()

0    0
1    0
2    0
3    0
4    0
Name: price, dtype: object

In [37]:
df_calendar['price']=df_calendar['price'].astype('float64') 

In [38]:
df_calendar['price']

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
5           0.0
6           0.0
7           0.0
8           0.0
9           0.0
10          0.0
11          0.0
12          0.0
13          0.0
14          0.0
15          0.0
16          0.0
17          0.0
18          0.0
19          0.0
20          0.0
21          0.0
22          0.0
23          0.0
24          0.0
25          0.0
26          0.0
27          0.0
28          0.0
29          0.0
           ... 
1308860    65.0
1308861    65.0
1308862    65.0
1308863    65.0
1308864    62.0
1308865    62.0
1308866    62.0
1308867     0.0
1308868     0.0
1308869     0.0
1308870    65.0
1308871    62.0
1308872    62.0
1308873    62.0
1308874    62.0
1308875    62.0
1308876    62.0
1308877    62.0
1308878    62.0
1308879    62.0
1308880     0.0
1308881     0.0
1308882     0.0
1308883     0.0
1308884     0.0
1308885     0.0
1308886     0.0
1308887     0.0
1308888     0.0
1308889     0.0
Name: price, Length: 130

In [39]:
df_calendar['date'].head()

0    2017-09-05
1    2017-09-04
2    2017-09-03
3    2017-09-02
4    2017-09-01
Name: date, dtype: object

In [40]:
df_calendar['date'].value_counts()

2017-03-15    3586
2016-12-11    3586
2017-06-05    3586
2017-01-16    3586
2017-07-19    3586
2017-09-05    3586
2017-01-22    3586
2017-05-18    3586
2017-02-18    3586
2016-11-03    3586
2017-01-08    3586
2017-03-29    3586
2017-04-20    3586
2017-08-21    3586
2016-09-11    3586
2016-10-03    3586
2016-09-10    3586
2017-04-07    3586
2017-08-31    3586
2017-02-12    3586
2017-08-11    3586
2017-04-13    3586
2017-01-13    3586
2016-12-16    3586
2017-01-30    3586
2016-11-05    3586
2017-04-19    3586
2017-05-24    3586
2017-01-01    3586
2016-09-22    3586
              ... 
2017-07-12    3586
2017-03-12    3586
2017-03-21    3586
2017-07-22    3586
2017-07-03    3586
2017-09-01    3586
2017-07-30    3586
2017-01-05    3586
2017-06-03    3586
2017-05-26    3586
2016-09-20    3586
2017-06-10    3586
2017-08-14    3586
2017-02-13    3586
2017-07-10    3586
2017-02-05    3586
2016-09-16    3586
2017-06-23    3586
2017-02-27    3586
2017-04-28    3586
2016-09-29    3586
2017-06-26  

In [41]:
df_calendar['date'].isnull().sum() #check if there is a NULL for date

0

In [43]:
#change the date to datetime type
df_calendar['date'] = pd.to_datetime(df_calendar['date'],format='%Y-%m-%d')

In [44]:
df_calendar.dtypes

listing_id             int64
date          datetime64[ns]
available             object
price                float64
dtype: object

In [45]:
df_calendar['available'] = df_calendar['available'].map({'t':True,'f':False})

In [46]:
df_calendar.dtypes

listing_id             int64
date          datetime64[ns]
available               bool
price                float64
dtype: object